In [ ]:
from transduction import LazyPrecoverNFA, FST, EPSILON
from transformers import AutoTokenizer
from transduction.lm.statelm import HfTokenizerVocab
from transduction.viz import visualize_automaton

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=False)
_decode = HfTokenizerVocab(tokenizer).decode

In [21]:
def bpe_fst(decode, token_ids=None, drop=frozenset()):
    """Build a BPE FST from a subset of token IDs."""
    if token_ids is None: token_ids = list(range(len(decode)))
    m = FST()
    m.add_start(())
    for i in token_ids:
        x = decode[i]
        if x in drop:
            continue
        bx = tuple(x)
        for j in range(len(bx)):
            m.add_arc(bx[:j], EPSILON, bx[j], bx[:j+1])
        m.add_arc(bx, i, EPSILON, ())
    m.add_stop(())
    return m.renumber()

In [77]:
fst = bpe_fst(_decode[:6000])

In [78]:
#fst

In [79]:
target = "The quick brown fox jumps over the lazy dog.".encode()
#target = "The ".encode()

In [80]:
nfa = LazyPrecoverNFA(fst, target)
tmp = nfa.materialize().renumber().epsremove()
#tmp

In [81]:
len(tmp.states), len(list(tmp.arcs()))

(10652, 63018383)

In [82]:
len(fst.A), len(fst.B)

(6001, 257)